In [1]:
# od pair
from env import env
import pandas as pd
import random as rd

# here requests
from haversine import haversine, Unit
import flexpolyline as fp
import urllib.request
import json, sys

# map plot
from bs4 import BeautifulSoup as bs
import gmplot

### Get od pair

In [17]:
# try later: https://geobgu.xyz/py/shapely.html
"""
df = pd.read_csv('results/bike-results/result.csv')

lat = df['station_lat'].to_list()
lng = df['station_lng'].to_list()

rd.seed(31)
x = rd.sample(range(0,2191),2)

originCoord = (lat[x[0]],lng[x[0]])
origin = f'{originCoord[0]},{originCoord[1]}'

destinationCoord = (lat[x[1]],lng[x[1]])
destination = f'{destinationCoord[0]},{destinationCoord[1]}'

print(origin)
print(originCoord)
print(destination)
print(destinationCoord)
"""


41.9106,-87.6494
(41.9106, -87.6494)
41.76,-87.58
(41.76, -87.58)


In [2]:
filtered = pd.read_csv('datasets/new_bike/filtered_dataset_gas.csv')
odDf = pd.read_csv('datasets/new_bike/set_id_coord.csv')

In [9]:
# coletar par od qualquer para teste

ind = 0 # filtered route index
od = odDf.loc[odDf['setID'] == filtered.iloc[ind]['setID']]

origin = od['departure_coord'].to_string(index=False).split(":")
origin = f"{origin[1]},{origin[0]}"

destination = od['arrival_coord'].to_string(index=False).split(":")
destination = f"{destination[1]},{destination[0]}"

float_origin = list(map(lambda x: float(x),origin.split(',')))
float_destination = list(map(lambda x: float(x),destination.split(',')))

print(origin)
print(destination)
print(float_origin)
print(float_destination)

51.533778,-0.2677
51.57419,-0.46799
[51.533778, -0.2677]
[51.57419, -0.46799]


### Here requests

Car co2 and fuel consumption link: https://developer.here.com/documentation/routing-api/dev_guide/topics/use-cases/calculate-fuel-consumption.html

In [4]:
def route_midpoint(route_length,zipList):#,latList,lngList):
    middle = delta = route_length/2 # route length was hereResponse['routes'][0]['sections'][0]['summary']['length']/2
    #zipList = list(zip(latList,lngList))
    before = after = None
    dist = 0

    for i in range(len(zipList)-1):
        dist += haversine(zipList[i],zipList[i+1], unit=Unit.METERS)
        sub = abs(middle-dist)
        if sub < delta:
            delta = sub
            before = zipList[i+1]
        if sub > delta:
            after = zipList[i+1]
            break

    midpointCoord = ((before[0]+after[0])/2,(before[1]+after[1])/2)
    midpoint = f'{midpointCoord[0]},{midpointCoord[1]}'

    return midpoint,midpointCoord

# get closest bike share point on coordinate
def closest(coord: list): #### need refactor!!
    data = pd.read_csv('results/bike-results/result.csv')
    dist = sys.float_info.max
    waypoint = ()
    for ind,row in data.iterrows():
        distTest = haversine(coord,(row['station_lat'],row['station_lng']))
        if distTest < dist:
            dist = distTest
            waypoint = (row['station_lat'],row['station_lng'])
    return waypoint

In [5]:
def car_routing(strCoord,maxAlternatives=0):
    query = f'destination={strCoord[1]}&origin={strCoord[0]}'
    hereKey = env('HERE_API_KEY')
    
    fuelConsumption = '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1'

    url = 'https://router.hereapi.com/v8/routes?transportMode=%s%s&%s&alternatives=%d&return=summary,polyline&apikey=%s' % ('car',fuelConsumption,query,maxAlternatives,hereKey)
    response = urllib.request.urlopen(url).read().decode('utf-8')
    return [json.loads(response)]


def transit_routing(strCoord,maxAlternatives=0):
    query = f'destination={strCoord[1]}&origin={strCoord[0]}'
    hereKey = env('HERE_API_KEY')

    # change fuel consumption for transport if possible
    #fuelConsumption = '&fuel[type]=diesel&fuel[freeflowspeedtable]=0,0,60,0.1,80,0.12&fuel[trafficspeedtable]=0,0,60,0.1,80,0.12&fuel[additionalconsumption]=1.8&fuel[ascent]=0.1'

    # multimodal here url
    # transposrt types:
    # vehicle
    # transit
    # taxi
    # rented
    
    url = 'https://intermodal.router.hereapi.com/v8/routes?%s&transit[enable]=entireRoute&rented[enable]=&taxi[enable]=&vehicle[enable]=&alternatives=%d&return=travelSummary,polyline&apiKey=%s' % (query,maxAlternatives,hereKey)
    response = urllib.request.urlopen(url).read().decode('utf-8')
    return [json.loads(response)]


# for pedestrian: https://developer.here.com/documentation/routing-api/dev_guide/topics/pedestrian-defined.html
def bicycle_routing(strCoord,floatCoord,maxAlternatives=1):
    hereKey = env('HERE_API_KEY')

    newOrigin = closest(floatCoord[0]) # data
    newDestination = closest(floatCoord[1]) # data

    mode = ['pedestrian','bicycle','pedestrian']

    query = [
        f'destination={newOrigin[0]},{newOrigin[1]}&origin={strCoord[0]}',
        f'destination={newDestination[0]},{newDestination[1]}&origin={newOrigin[0]},{newOrigin[1]}',
        f'destination={strCoord[1]}&origin={newDestination[0]},{newDestination[1]}'
    ]

    responses = []

    for i in range(len(mode)):
        if mode[i] == 'pedestrian': altNum = 0
        else: altNum = maxAlternatives
        url = 'https://router.hereapi.com/v8/routes?transportMode=%s&%s&alternatives=%d&return=summary,polyline&apikey=%s' % (mode[i],query[i],altNum,hereKey)
        print(url) # ------------------
        response = urllib.request.urlopen(url).read().decode('utf-8')
        responses.append(json.loads(response))

    return responses

In [6]:
def response_reader(response,midpoint=False): # retorna polilinha das respostas
    polyline = []
    distance = 0
    duration = 0

    for section in response['routes'][0]['sections']:
        try:
            distance += section['summary']['length']
            duration += section['summary']['duration']
        except:
            distance += section['travelSummary']['length']
            duration += section['travelSummary']['duration']
        polyline += fp.decode(section['polyline'])

    if midpoint: return duration,distance,polyline,route_midpoint(distance,polyline)
    return duration,distance,polyline

In [14]:
coords = (origin,destination)
float_coords = (float_origin,float_destination)

response_car = car_routing(coords) # full car
response_transit = transit_routing(coords) # full transit
#responses_bicycle = bicycle_routing(coords,float_coords) # full bicycle

In [15]:
duration_car,distance_car,polyline_car,car_midpoint = response_reader(*response_car,True)
duration_transit,distance_transit,polyline_transit,transit_midpoint = response_reader(*response_transit,True)

In [16]:
response_car_transit_1 = car_routing((origin,car_midpoint[0])) # car_transit_1
response_car_transit_2 = transit_routing((car_midpoint[0],destination)) # car_transit_2

response_transit_car_1 = transit_routing((origin,transit_midpoint[0])) # transit_car_1
response_transit_car_2 = car_routing((transit_midpoint[0],destination)) # transit_car_2

In [18]:
duration_ct1,distance_ct1,polyline_ct1 = response_reader(*response_car_transit_1)
duration_ct2,distance_ct2,polyline_ct2 = response_reader(*response_car_transit_2)

duration_tc1,distance_tc1,polyline_tc1 = response_reader(*response_transit_car_1)
duration_tc2,distance_tc2,polyline_tc2 = response_reader(*response_transit_car_2)

print(duration_ct1+duration_ct2,distance_ct1+distance_ct2)
print(duration_tc1+duration_tc2,distance_tc1+distance_tc2)

4468 19168
2642 20627


In [45]:
# original car ++
# car
# transit
# car + transit
# transit + car
#og_polyline = filtered.iloc[ind]['coordinates'].replace(':',',').split(';')
#og_treated_polyline = []
#for i in og_polyline:
#    j = i.split(',')
#    og_treated_polyline.append((float(j[0]),float(j[1])))

#print(og_treated_polyline)

[(-0.2677, 51.533778), (-0.2677, 51.53326), (-0.26763, 51.53323), (-0.26758, 51.53319), (-0.26758, 51.53314), (-0.26762, 51.5331), (-0.26769, 51.53308), (-0.26775, 51.53307), (-0.26784, 51.5331), (-0.26787, 51.53315), (-0.26848, 51.53298), (-0.26855, 51.53296), (-0.26928, 51.53268), (-0.26943, 51.53256), (-0.2695, 51.53248), (-0.26963, 51.53235), (-0.27, 51.53196), (-0.27007, 51.53189), (-0.27052, 51.53135), (-0.2708, 51.53108), (-0.27069, 51.53103), (-0.27065, 51.53097), (-0.27063, 51.53093), (-0.27064, 51.5309), (-0.27067, 51.53087), (-0.27071, 51.53084), (-0.27084, 51.5308), (-0.27093, 51.5308), (-0.271, 51.53082), (-0.27108, 51.53089), (-0.27143, 51.53075), (-0.27174, 51.53061), (-0.27243, 51.53059), (-0.27277, 51.53058), (-0.274, 51.53055), (-0.27529, 51.53052), (-0.27729, 51.53047), (-0.27755, 51.53047), (-0.27796, 51.53048), (-0.27853, 51.5306), (-0.27856, 51.53053), (-0.27865, 51.53047), (-0.27877, 51.53045), (-0.27872, 51.53015), (-0.27872, 51.53008), (-0.27869, 51.5295), (-0.

In [13]:
def map_plot(origin,og_polyline,new_polyline_list,color_list,fileName):
    origin_coord = origin.split(',')
    outputMap = gmplot.GoogleMapPlotter(float(origin_coord[0]),float(origin_coord[1]), 12)
    mapUrl = f'results/map_plot/{fileName}.html'

    lats = [t[1] for t in og_polyline]
    lngs = [t[0] for t in og_polyline]
    
    outputMap.plot(lats,lngs, color="black", edge_width=3)

    for i in range(len(new_polyline_list)):

        new_lats = [t[0] for t in new_polyline_list[i]]
        new_lngs = [t[1] for t in new_polyline_list[i]]

        outputMap.plot(new_lats, new_lngs, color=color_list[i], edge_width=3)

    outputMap.draw(mapUrl)

    page = open(mapUrl)
    soup = bs(page.read(),"html.parser")
    srcKey  = f"https://maps.googleapis.com/maps/api/js?key={env('GOOGLE_MAPS_KEY')}&libraries=visualization&sensor=true_or_false"
    line = soup.find('script',type="text/javascript")
    line['src'] = srcKey

    with open(mapUrl, "wb") as f_output:
        f_output.write(soup.prettify("utf-8")) 

In [70]:
og_polyline = filtered.iloc[ind]['coordinates'].replace(':',',').split(';')
og_treated_polyline = []
for i in og_polyline:
    j = i.split(',')
    og_treated_polyline.append((float(j[0]),float(j[1])))

In [78]:
map_plot(origin,og_treated_polyline,[polyline_car],["blue"],"full_car")
map_plot(origin,og_treated_polyline,[polyline_transit],["red"],"full_transit")

map_plot(origin,og_treated_polyline,[polyline_ct1,polyline_ct2],['blue','red'],'car_transit')
map_plot(origin,og_treated_polyline,[polyline_tc1,polyline_tc2],['red','blue'],'transit_car')

## Multimodal Test

In [47]:
#bicycle routing test: ped - bicycle - ped
#response = bicycle_routing((origin,destination),(originCoord,destinationCoord))